In [ ]:
%%capture
!apt-get install maven -qq

In [ ]:
%%capture
!git clone --recurse-submodules https://github.com/castorini/anserini.git
%cd anserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true

In [ ]:
#import the necessary data (i am not sure this is the latest data)
!mkdir collections/fever
!mkdir indexes/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/wiki-pages.zip -P collections/fever
!unzip collections/fever/wiki-pages.zip -d collections/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/train.jsonl -P collections/fever
!wget https://s3-eu-west-1.amazonaws.com/fever.public/paper_dev.jsonl -P collections/fever

--2021-05-27 19:00:13--  https://s3-eu-west-1.amazonaws.com/fever.public/wiki-pages.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.1.19
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.1.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1713485474 (1.6G) [application/zip]
Saving to: ‘collections/fever/wiki-pages.zip’

wiki-pages.zip      100%[===================>]   1.60G  28.2MB/s    in 61s     

2021-05-27 19:01:15 (26.7 MB/s) - ‘collections/fever/wiki-pages.zip’ saved [1713485474/1713485474]

Archive:  collections/fever/wiki-pages.zip
  inflating: collections/fever/license.html  
   creating: collections/fever/__MACOSX/
  inflating: collections/fever/__MACOSX/._license.html  
   creating: collections/fever/wiki-pages/
  inflating: collections/fever/wiki-pages/wiki-001.jsonl  
   creating: collections/fever/__MACOSX/wiki-pages/
  inflating: collections/fever/__MACOSX/wiki-pages/._wiki-001.jsonl  


In [ ]:
#Indexing of the wikipedia dump (extracted from wiki_zip)
!sh target/appassembler/bin/IndexCollection \
 -collection FeverParagraphCollection -generator DefaultLuceneDocumentGenerator \
 -threads 9 -input collections/fever/wiki-pages \
 -index indexes/fever/lucene-index-fever-paragraph -storePositions -storeDocvectors -storeRaw 

2021-05-27 19:03:01,435 INFO  [main] index.IndexCollection (IndexCollection.java:641) - Setting log level to INFO
2021-05-27 19:03:01,438 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Starting indexer...
2021-05-27 19:03:01,438 INFO  [main] index.IndexCollection (IndexCollection.java:645) - ============ Loading Parameters ============
2021-05-27 19:03:01,439 INFO  [main] index.IndexCollection (IndexCollection.java:646) - DocumentCollection path: collections/fever/wiki-pages
2021-05-27 19:03:01,440 INFO  [main] index.IndexCollection (IndexCollection.java:647) - CollectionClass: FeverParagraphCollection
2021-05-27 19:03:01,440 INFO  [main] index.IndexCollection (IndexCollection.java:648) - Generator: DefaultLuceneDocumentGenerator
2021-05-27 19:03:01,441 INFO  [main] index.IndexCollection (IndexCollection.java:649) - Threads: 9
2021-05-27 19:03:01,442 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Stemmer: porter
2021-05-27 19:03:01,442 INFO  [main] ind

In [ ]:
!tar -cvf "/content/anserini/indexes/fever/lucene-index-fever-paragraph.tar" "/content/anserini/indexes/fever/lucene-index-fever-paragraph" #Tar operation for exporting the index and make it reusable
# Only one indexing will be enough we will use the tar file after that use the code below to untar it and equate to the LucuneDir
#!tar xvfz lucene-index-fever-paragraph.tar

tar: Removing leading `/' from member names
/content/anserini/indexes/fever/lucene-index-fever-paragraph/
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.nvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1_Lucene50_0.doc
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_7.nvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_7.fnm
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_5.fdt
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_0.tvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_8_Lucene50_0.pos
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_7.nvm
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_4.nvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_0.nvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1_Lucene50_0.tim
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_2.fnm
/content/anserini/indexes/fever/lucene-index-

In [ ]:
luceneDir = 'indexes/fever/lucene-index-fever-paragraph'

In [ ]:
#%%capture
!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

In [ ]:
query_processed = 'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'
keywords = '2019-nCoV, SARS-CoV-2, COVID-19, symptoms, hospitalization'

In [ ]:
import json

searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query_processed + '. ' + keywords,k=50) # We can retrieve the documents according to concatanated string of query and keywords we can change this setup.
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}

In [ ]:
hits[0].raw

"0\tThe SARS conspiracy theory began to emerge during the severe acute respiratory syndrome -LRB- SARS -RRB- outbreak in China in the spring of 2003 , when Sergei Kolesnikov , a Russian scientist and a member of the Russian Academy of Medical Sciences , first publicized his claim that the SARS coronavirus is a synthesis of measles and mumps .\tsevere acute respiratory syndrome\tsevere acute respiratory syndrome\tChina\tChina\tSergei Kolesnikov\tSergei Kolesnikov (scientist)\tSARS coronavirus\tSARS coronavirus\tmeasles\tmeasles\tmumps\tmumps\tcoronavirus\tcoronavirus\n1\tAccording to Kolesnikov , this combination can not be formed in the natural world and thus the SARS virus must have been produced under laboratory conditions .\n2\tAnother Russian scientist , Nikolai Filatov , head of Moscow 's epidemiological services , had earlier commented that the SARS virus was probably man-made .\tNikolai Filatov\tNikolai Filatov\tMoscow\tMoscow\tepidemiological\tepidemiology\n3\t\n4\tHowever , in

In [ ]:
hits[0].lucene_document.fields.toString()

"[stored,indexed,tokenized,omitNorms,indexOptions=DOCS<id:SARS_conspiracy_theory>, stored<raw:0\tThe SARS conspiracy theory began to emerge during the severe acute respiratory syndrome -LRB- SARS -RRB- outbreak in China in the spring of 2003 , when Sergei Kolesnikov , a Russian scientist and a member of the Russian Academy of Medical Sciences , first publicized his claim that the SARS coronavirus is a synthesis of measles and mumps .\tsevere acute respiratory syndrome\tsevere acute respiratory syndrome\tChina\tChina\tSergei Kolesnikov\tSergei Kolesnikov (scientist)\tSARS coronavirus\tSARS coronavirus\tmeasles\tmeasles\tmumps\tmumps\tcoronavirus\tcoronavirus\n1\tAccording to Kolesnikov , this combination can not be formed in the natural world and thus the SARS virus must have been produced under laboratory conditions .\n2\tAnother Russian scientist , Nikolai Filatov , head of Moscow 's epidemiological services , had earlier commented that the SARS virus was probably man-made .\tNikolai 